In [1]:
# Useful for debugging
%load_ext autoreload
%autoreload 2

# Xopt class, GPT CBETA DC GUN example

This is the class method for running Xopt.

In [2]:
from xopt import Xopt

In [9]:
import yaml

# Make a proper input file. 

YAML="""
xopt: {output_path: null, verbose: true,
  algorithm: cnsga}
  
algorithm:
  name: cnsga
  options: {max_generations: 16, population_size: 8, crossover_probability: 0.9, mutation_probability: 1.0,
  selection: auto, verbose: true, population: null}

simulation:
  name: gpt_with_distgen
  evaluate: gpt.evaluate.evaluate_gpt_with_distgen

# Note that you can call another file in the top level group:
vocs: ../simulations/cbeta_dcgun/cbeta_dcgun_vocs.json

"""
config = yaml.safe_load(YAML)

In [10]:
# Create object
X = Xopt(config=config)

# Show config
X

AssertionError: path does not exist: /Users/chrisonian/Code/GitHub/xopt/examples/simulations/cbeta_dcgun/cbeta_dcgun_vocs.json

In [4]:
# Check vocs
X.vocs

{'constants': {'gun_voltage': 375},
 'constraints': {'end_std_t': ['LESS_THAN', 0.0015]},
 'description': 'Test case',
 'name': 'CBETA DC GUN',
 'objectives': {'end_norm_emitt_x': 'MINIMIZE', 'end_qbunch': 'MAXIMIZE'},
 'simulation': 'gpt_with_distgen',
 'templates': {'distgen_input_file': '/Users/chrisonian/Code/GitHub/xopt/examples/cbeta.dcgun/template/cbeta.dcgun.distgen.in.json',
  'gpt_input_file': '/Users/chrisonian/Code/GitHub/xopt/examples/cbeta.dcgun/template/cbeta.dcgun.gpt.in'},
 'variables': {'beam:params:sigma_xy:value': [0, 5],
  'beam:params:total_charge:value': [0, 500],
  'sol_1_current': [0, 5]},
 'linked_variables': None}

In [5]:
# Check random inputs
X.random_inputs()

{'beam:params:sigma_xy:value': 2.664319481693366,
 'beam:params:total_charge:value': 73.60757260692102,
 'sol_1_current': 1.9772798435379442,
 'gun_voltage': 375}

In [6]:
%%time
# Do a random evaluate to check that everything will run
x=X.random_evaluate()
print(x)

/Users/chrisonian/Code/miniconda3/envs/devel/lib/python3.7/site-packages/pint/quantity.py:221: BehaviorChangeWarning: The way Pint handles NumPy operations has changed with the
implementation of NEP 18. Unimplemented NumPy operations will now fail instead of making
assumptions about units. Some functions, eg concat, will now return Quanties with units, where
they returned ndarrays previously. See https://github.com/hgrecco/pint/pull/905.

To hide this warning, wrap your first creation of an array Quantity with
warnings.catch_warnings(), like the following:

import numpy as np
import warnings
from pint import Quantity

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    Quantity([])

To disable the new behavior, see
https://www.numpy.org/neps/nep-0018-array-function-protocol.html#implementation

  warnings.warn(array_function_change_msg, BehaviorChangeWarning)


{'error': False, 'end_std_x': 0.0038227064216930207, 'end_std_y': 0.003810873576023852, 'end_qbunch': 8.190617220656904e-11, 'end_norm_emitt_x': 1.965279942654443e-06, 'end_norm_emitt_y': 1.952997519487586e-06, 'end_std_t': 1.703792758213823e-11, 'fingerprint': '05dddf02e78239eea16899e91fa43232'}
CPU times: user 64.4 ms, sys: 21.2 ms, total: 85.6 ms
Wall time: 1.39 s


In [7]:
X.vocs['constraints']

{'end_std_t': ['LESS_THAN', 0.0015]}

In [8]:
# These are the algorithm options
X.algorithm['options']

{'max_generations': 16,
 'population_size': 8,
 'crossover_probability': 0.9,
 'mutation_probability': 1.0,
 'selection': 'auto',
 'verbose': True,
 'seed': None}

In [9]:
# These are the options in the evaluate function
X.simulation['options']

{'archive_path': None,
 'merit_f': None,
 'distgen_input_file': '$HOME/Code/GitHub/xopt/examples/cbeta.dcgun/template/cbeta.dcgun.distgen.in.json',
 'gpt_input_file': '$HOME/Code/GitHub/xopt/examples/cbeta.dcgun/template/cbeta.dcgun.gpt.in'}

# Run CNSGA using processes or threads

In [ ]:
from concurrent.futures import ProcessPoolExecutor as PoolExecutor
#from concurrent.futures import ThreadPoolExecutor as PoolExecutor
executor = PoolExecutor()

In [ ]:
X.run_cnsga(executor=executor)

In [ ]:
# Check for errors
X.population['errors']

# Write this configuration

In [10]:
X.save('test.yaml')

Config written as YAML to test.yaml


In [11]:
!cat test.yaml

xopt: {output_path: null, verbose: true, algorithm: cnsga}
algorithm:
  name: cnsga
  options: {max_generations: 16, population_size: 8, crossover_probability: 0.9, mutation_probability: 1.0,
    selection: auto, verbose: true, seed: null}
  max_generations: 16
  population_size: 8
  crossover_probability: 0.9
  mutation_probability: 1.0
  selection: auto
  verbose: true
  seed: null
simulation: {name: gpt_with_distgen, evaluate: gpt.evaluate.evaluate_gpt_with_distgen}
vocs:
  constants: {gun_voltage: 375}
  constraints:
    end_std_t: [LESS_THAN, 0.0015]
  description: Test case
  name: CBETA DC GUN
  objectives: {end_norm_emitt_x: MINIMIZE, end_qbunch: MAXIMIZE}
  simulation: gpt_with_distgen
  templates: {distgen_input_file: /Users/chrisonian/Code/GitHub/xopt/examples/cbeta.dcgun/template/cbeta.dcgun.distgen.in.json,
    gpt_input_file: /Users/chrisonian/Code/GitHub/xopt/examples/cbeta.dcgun/template/cbeta.dcgun.gpt.in}
  variables:
    beam:params:sigma_xy:value: [0, 5]
    beam:pa

# Run with MPI

In [16]:
!mpirun -n 4 python -m mpi4py.futures -m xopt.mpi.run test.yaml

  _   
                | |  
__  _____  _ __ | |_ 
\ \/ / _ \| '_ \| __|
 >  < (_) | |_) | |_ 
/_/\_\___/| .__/ \__|
          | |        
          |_|        

_________________________________
Parallel execution with 4 workers
Loading /Users/chrisonian/Code/GitHub/xopt/examples/xopt_class/test.yaml as YAML 

{'max_generations': 16, 'population_size': 8, 'crossover_probability': 0.9, 'mutation_probability': 1.0, 'selection': 'auto', 'verbose': True, 'seed': None}



 ▄████▄   ███▄    █   ██████   ▄████  ▄▄▄      
▒██▀ ▀█   ██ ▀█   █ ▒██    ▒  ██▒ ▀█▒▒████▄    
▒▓█    ▄ ▓██  ▀█ ██▒░ ▓██▄   ▒██░▄▄▄░▒██  ▀█▄  
▒▓▓▄ ▄██▒▓██▒  ▐▌██▒  ▒   ██▒░▓█  ██▓░██▄▄▄▄██ 
▒ ▓███▀ ░▒██░   ▓██░▒██████▒▒░▒▓███▀▒ ▓█   ▓██▒
░ ░▒ ▒  ░░ ▒░   ▒ ▒ ▒ ▒▓▒ ▒ ░ ░▒   ▒  ▒▒   ▓▒█░
  ░  ▒   ░ ░░   ░ ▒░░ ░▒  ░ ░  ░   ░   ▒   ▒▒ ░
░           ░   ░ ░ ░  ░  ░  ░ ░   ░   ░   ▒   
░ ░               ░       ░        ░       ░  ░
░                                              


Continous Non-dominated Sorting Genetic Algor

# Dask 

In [ ]:
from dask.distributed import Client
executor = Client()
executor

In [ ]:
# Wait a few seconds for the Dask cluster to start
from time import sleep
sleep(5)

In [ ]:
# Increase the number of generations
X.algorithm['options']['max_generations'] = 7
X.run_cnsga(executor)

In [ ]:
X.population

# Cleanup

In [ ]:
!rm -r dask-worker-space

In [ ]:
!rm test.yaml